# Import

In [1]:
# !pip install rdkit
# !pip install albumentations

In [2]:
import random
import os

from tqdm import tqdm

import numpy as np
import pandas as pd
pd.set_option('mode.chained_assignment', None)

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from rdkit import DataStructs
from rdkit.Chem import PandasTools, AllChem

In [3]:
import os
import random
import numpy as np
import torch

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [4]:
class CFG:
    SEED = 0
    IMG_SIZE = 224
    BATCH_SIZE = 64
    EPOCHS = 1024
    LEARNING_RATE = 0.003

<br></br>

# Data Load

In [5]:
train_df = pd.read_csv('./data/train.csv')
test_df  = pd.read_csv('./data/test.csv')

In [6]:
train_df.head()

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.259,117.37
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.172,73.47
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.585,62.45
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.475,92.60
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.337,42.43


<br></br>

# EDA

In [7]:
# train_df.describe()

In [8]:
# num_features = train_df.columns[train_df.dtypes!='object'].tolist()
# for i,col in enumerate(num_features):

#     fig = plt.figure(figsize=(15,7))
#     fig.add_subplot(121)
#     sns.histplot(train_df[col],bins=20)
#     plt.grid()

#     fig.add_subplot(122)
#     sns.histplot(np.log(train_df[col]+1e-3),bins=20)
#     plt.grid()

#     plt.suptitle('[{}/{}] {}'.format(i+1,len(num_features),col))
#     plt.tight_layout()
#     plt.show()

# # -> ['Molecular_Weight','Molecular_PolarSurfaceArea']

In [9]:
# cols = ['AlogP','Molecular_Weight','Num_H_Acceptors','Num_H_Donors','Num_RotatableBonds','LogD','Molecular_PolarSurfaceArea']
# for col in cols:
#     print(col)
#     plt.figure(figsize=(15,7))
#     sns.scatterplot(x=train_df[col],y=train_df['HLM'])
#     plt.grid()
#     plt.show()

In [10]:
# cols = ['Num_H_Acceptors','Num_H_Donors','Num_RotatableBonds']
# for col in cols:
#     print(col)
#     plt.figure(figsize=(15,7))
#     sns.boxplot(x=train_df[col],y=train_df.MLM)
#     plt.show()

In [11]:
# train_df.nunique()

In [12]:
# lists = sorted(train_df['Num_H_Acceptors'].unique())
# for v in lists:
#     print('########',v)
#     d = train_df[train_df['Num_H_Acceptors']==v]
    
#     cols = ['AlogP','Molecular_Weight','Num_H_Acceptors','Num_H_Donors','Num_RotatableBonds','LogD','Molecular_PolarSurfaceArea']
#     for col in cols:
#         print(col)
#         plt.figure(figsize=(15,7))
#         sns.scatterplot(x=d[col],y=d['HLM'])
#         plt.grid()
#         plt.show()

<br></br>

# Pre-Processing

<br>

## Set target range to [0,100]

In [13]:
targets = ['MLM','HLM']
for t in targets:
    train_df[t] = [0 if x<0 else
                   100 if x>100 else
                   x for x in train_df[t]]

<br>

## Make molecule features

In [14]:
# Molecule to MorganFingerprint
def mol2fp(mol):
    fp = AllChem.GetHashedMorganFingerprint(mol, 6, nBits=4096)
    ar = np.zeros((1,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(fp, ar)
    return ar

In [15]:
# (1) SMILES를 통해 Molecule(분자구조) 생성
PandasTools.AddMoleculeColumnToFrame(train_df,'SMILES','Molecule')
PandasTools.AddMoleculeColumnToFrame(test_df ,'SMILES','Molecule')

In [16]:
from sklearn.feature_selection import VarianceThreshold

In [17]:
# (2) Morgan Fingerprint column 추가
train_df["FPs"] = train_df.Molecule.apply(mol2fp)
test_df ["FPs"] = test_df .Molecule.apply(mol2fp)

In [18]:
# (3) Morgan Fingerprint 중, variance가 0.05보다 작은 컬럼들을 지우기
feature_select = VarianceThreshold(threshold=0.05)

# 일부사용
tr_fps_selected = feature_select.fit_transform(np.stack(train_df['FPs']))
te_fps_selected = feature_select.transform(np.stack(test_df['FPs']))
print(len(tr_fps_selected))

# # 전체사용
# tr_fps_selected = np.stack(train_df['FPs'])
# te_fps_selected = np.stack(test_df ['FPs'])

fps_names = ['fps'+str(i+1) for i in range(tr_fps_selected.shape[1])]

train_df = pd.concat([
    train_df.drop('FPs',axis=1),
    pd.DataFrame(tr_fps_selected,columns=fps_names),
],axis=1)

test_df = pd.concat([
    test_df.drop('FPs',axis=1),
    pd.DataFrame(te_fps_selected,columns=fps_names),
],axis=1)

3498


In [19]:
# train_df['fps_raw_sum'] = np.sum(train_df[fps_names].values,axis=1)
# test_df ['fps_raw_sum'] = np.sum(test_df [fps_names].values,axis=1)

In [20]:
# fps_dummy_names = []
# for col in tqdm(fps_names):
#     train_df[f'{col}_dmy'] = np.where(train_df[col]==0,0,1)
#     test_df [f'{col}_dmy'] = np.where(test_df [col]==0,0,1)
#     fps_dummy_names.append(f'{col}_dmy')

In [21]:
# train_df['fps_dummy_sum'] = np.sum(train_df[fps_names].values,axis=1)
# test_df ['fps_dummy_sum'] = np.sum(test_df [fps_names].values,axis=1)

In [22]:
# 사용할 column만 추출
features = ['AlogP','Molecular_Weight','Num_H_Acceptors','Num_H_Donors',
            'Num_RotatableBonds','LogD','Molecular_PolarSurfaceArea']
fps_features = [col for col in train_df.columns if col.find('fps')==0]
smiles_feature = 'SMILES'
targets  = ['MLM','HLM']

train_df = train_df[features+fps_features+[smiles_feature]+targets]
test_df  = test_df[features+fps_features+[smiles_feature]]

In [23]:
train_df.shape

(3498, 261)

<br>

## Imputaion

In [24]:
from sklearn.impute import SimpleImputer

In [25]:
def null_check(data):
    d = data.copy()
    null_info = d.isnull().sum()
    null_info = null_info[null_info!=0]
    display(null_info)

In [26]:
print('> train')
null_check(train_df)

print('> test')
null_check(test_df)

> train


AlogP    2
dtype: int64

> test


AlogP    1
dtype: int64

In [27]:
null_features = ['AlogP']

imputer = SimpleImputer(missing_values=np.nan, strategy='median')
train_df[null_features] = imputer.fit_transform(train_df[null_features])
test_df [null_features] = imputer.transform(test_df[null_features])

In [28]:
print('> train')
null_check(train_df)

print('> test')
null_check(test_df)

> train


Series([], dtype: int64)

> test


Series([], dtype: int64)

<br>

## Train Test Split

In [29]:
from sklearn.model_selection import train_test_split

In [30]:
tr_df, va_df = train_test_split(train_df,test_size=0.1,shuffle=True,random_state=CFG.SEED)
te_df = test_df.copy()

In [31]:
len(tr_df), len(va_df), len(te_df)

(3148, 350, 483)

<br>

## Scaling

In [32]:
from sklearn.preprocessing import MinMaxScaler

scaling_features = features#+fps_features
scalers = {}
for f in scaling_features:
    scaler = MinMaxScaler()
    tr_df[f] = scaler.fit_transform(np.array(tr_df[f]).reshape(-1,1))
    va_df[f] = scaler.transform(np.array(va_df[f]).reshape(-1,1))
    te_df[f] = scaler.transform(np.array(te_df[f]).reshape(-1,1))
    scalers[f] = scaler

<br>

## Interaction Term

In [33]:
import warnings
from tqdm import trange

def get_abs_corr(x,y):
    return np.abs(np.corrcoef(x,y))[0,1]

class InteractionTerm:
    def __init__(self):
        pass
    
    def fit(self,data,num_features,corr_cutoff=0.7):
        warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
        warnings.filterwarnings("ignore", category=RuntimeWarning)
        
        d = data.copy()
        self.interaction_list = []
        for i in trange(len(num_features),desc='fitting...'):
            for j in range(len(num_features)):
                if i>j:
                    col_i = num_features[i]
                    col_j = num_features[j]
                    
                    # 상관계수가 cutoff보다 큰 경우에는 interaction을 생성하지 않음
                    if (get_abs_corr(d[col_i]*d[col_j],d[col_i])>=corr_cutoff) | (get_abs_corr(d[col_i]*d[col_j],d[col_j])>=corr_cutoff):
                        pass
                    else:
                        self.interaction_list.append(f'{col_i}*{col_j}')
    
    def transform(self,data):
        d = data.copy()
        print('> the number of interaction term:',len(self.interaction_list))
        for interaction in self.interaction_list:
            col_i,col_j = interaction.split('*')
            d[interaction] = d[col_i]*d[col_j]
        return d
    
    def fit_transform(self,data,num_features,corr_cutoff=0.7):
        self.fit(data,num_features,corr_cutoff)
        return self.transform(data)

In [34]:
# num_features = features + fps_features

In [35]:
# interaction_maker = InteractionTerm()
# interaction_maker.fit(
#     data=tr_df,
#     num_features=num_features,
#     corr_cutoff=0.15,
# )
# tr_df = interaction_maker.transform(tr_df)
# va_df = interaction_maker.transform(va_df)
# te_df = interaction_maker.transform(te_df)

In [36]:
tr_df.shape

(3148, 261)

<br>

## Target Transformation

In [37]:
# for t in targets:
#     tr_df[t] = np.log(tr_df[t]+1e-3)
#     va_df[t] = np.log(va_df[t]+1e-3)

In [38]:
# def inverse_transform(x):
#     return torch.exp(x)-1e-3

In [41]:
from scipy.special import boxcox, inv_boxcox

def boxcox_transform(x):
    _lambda = 0.25
    return boxcox(x,_lambda)

def inverse_boxcox_transform(x):
    _lambda = 0.25
    return inv_boxcox(x,_lambda)

In [42]:
tr_df[targets] = tr_df[targets].apply(boxcox_transform)
va_df[targets] = va_df[targets].apply(boxcox_transform)

<br></br>

# Custom Dataset

In [43]:
from rdkit import Chem
from rdkit.Chem import Draw

class CustomDataset(Dataset):
    def __init__(self, data, targets, smiles, transforms=None, is_test=False):
        self.data = data.copy()
        self.targets = targets
        self.smiles = smiles
        self.transforms = transforms
        self.is_test = is_test
        
        self.smiles_features = []
        for s in tqdm(data[smiles].values):
            m = Chem.MolFromSmiles(s)
            img = Draw.MolToImage(m)#, size=(224,224))
            img = np.array(img)
            if self.transforms is not None:
                img = self.transforms(image=img)['image']
            self.smiles_features.append(img)
            
        if not self.is_test:
            self.target_features = self.data[self.targets].values
            self.num_features = self.data.drop(columns=targets+[smiles],axis=1).values
        else:
            self.num_features = self.data.drop(columns=[smiles],axis=1).values

    def __getitem__(self, index):
        if self.is_test:
            return (
                torch.Tensor(self.num_features[index]),
                torch.Tensor(self.smiles_features[index]),
            )
        else:
            return (
                torch.Tensor(self.num_features[index]),
                torch.Tensor(self.smiles_features[index]),
                torch.Tensor(self.target_features[index]),
            )
        
    def __len__(self):
        return len(self.data)

In [44]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

In [45]:
#  이미지 변환
transform = A.Compose([
    A.Resize(CFG.IMG_SIZE,CFG.IMG_SIZE),
    A.ToGray(p=1),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
    ToTensorV2(),
])

In [46]:
train_dataset = CustomDataset(tr_df, ['MLM','HLM'], 'SMILES', transform, False)
val_dataset   = CustomDataset(va_df, ['MLM','HLM'], 'SMILES', transform, False)
test_dataset  = CustomDataset(te_df, ['MLM','HLM'], 'SMILES', transform, True)

100%|██████████| 483/483 [00:04<00:00, 105.15it/s]


In [47]:
train_loader = DataLoader(train_dataset, batch_size=CFG.BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=0)
val_loader   = DataLoader(val_dataset  , batch_size=CFG.BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=0)
test_loader  = DataLoader(test_dataset , batch_size=CFG.BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=0)

In [49]:
# train_dataset.smiles_features[0][1]

In [50]:
# [feat for feat,img,target in train_loader][0]
# [img for feat,img in test_loader][0]

In [51]:
# import matplotlib.pyplot as plt

# _img = [img for feat,img,target in train_loader][0][0].numpy()
# plt.imshow(_img.T)

<br></br>

# Modeling

## Define Model

In [52]:
import torchvision.models as models

In [53]:
class MultiTaskModel(nn.Module):
    def __init__(self, feature_input_size, output_size, hidden_size, dropout_rate):
        super(MultiTaskModel, self).__init__()
        self.image_output_size = 500
        self.feature_output_size = 500
        self.dropout_rate = dropout_rate
        
        # # efficientnet
        # self.backbone = models.efficientnet_b0(pretrained=True)
        # self.backbone.classifier = nn.Sequential(
        #     nn.Dropout(p=0.2,inplace=True),
        #     nn.Linear(self.backbone.classifier[-1].in_features,self.image_output_size),
        # )
        
        # resnet
        self.backbone = models.resnet18(pretrained=True)
        self.backbone.fc = nn.Linear(self.backbone.fc.in_features,self.image_output_size)
        
        self.image_layer = nn.Sequential(
            self.backbone,
            #nn.BatchNorm1d(self.image_output_size),
            #nn.SiLU(),
            #nn.Dropout(self.dropout_rate),
        )
        
        self.feature_layer = nn.Sequential(
            nn.Linear(feature_input_size,hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.SiLU(),
            nn.Dropout(self.dropout_rate),
            nn.Linear(hidden_size,hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.SiLU(),
            nn.Dropout(self.dropout_rate),
            nn.Linear(hidden_size,hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.SiLU(),
            nn.Dropout(self.dropout_rate),
            nn.Linear(hidden_size,hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.SiLU(),
            nn.Dropout(self.dropout_rate),
            nn.Linear(hidden_size,self.feature_output_size),
        )
        
        fc_input_size = self.image_output_size+self.feature_output_size
        self.fc_1 = nn.Sequential(
            nn.Linear(fc_input_size,fc_input_size//2),
            nn.BatchNorm1d(fc_input_size//2),
            nn.SiLU(),
            nn.Dropout(self.dropout_rate),
            nn.Linear(fc_input_size//2,fc_input_size//4),
            nn.BatchNorm1d(fc_input_size//4),
            nn.SiLU(),
            nn.Dropout(self.dropout_rate),
            nn.Linear(fc_input_size//4,fc_input_size//8),
            nn.BatchNorm1d(fc_input_size//8),
            nn.SiLU(),
            nn.Dropout(self.dropout_rate),
            nn.Linear(fc_input_size//8,fc_input_size//16),
        )
        self.fc_2 = nn.Sequential(
            nn.Linear(fc_input_size,fc_input_size//2),
            nn.BatchNorm1d(fc_input_size//2),
            nn.SiLU(),
            nn.Dropout(self.dropout_rate),
            nn.Linear(fc_input_size//2,fc_input_size//4),
            nn.BatchNorm1d(fc_input_size//4),
            nn.SiLU(),
            nn.Dropout(self.dropout_rate),
            nn.Linear(fc_input_size//4,fc_input_size//8),
            nn.BatchNorm1d(fc_input_size//8),
            nn.SiLU(),
            nn.Dropout(self.dropout_rate),
            nn.Linear(fc_input_size//8,fc_input_size//16),
        )
        
        self.fc = nn.Sequential(
            nn.Linear(2*(fc_input_size//16),fc_input_size//16),
            nn.BatchNorm1d(fc_input_size//16),
            nn.SiLU(),
            nn.Dropout(self.dropout_rate),
            nn.Linear(fc_input_size//16,fc_input_size//32),
            nn.BatchNorm1d(fc_input_size//32),
            nn.SiLU(),
            nn.Dropout(self.dropout_rate),
            nn.Linear(fc_input_size//32,output_size),
        )
        
    def forward(self, image, feature):
        image_output = self.image_layer(image)
        feature_output = self.feature_layer(feature)
        combined = torch.cat((image_output,feature_output),dim=1)
        output1 = self.fc_1(combined)
        output2 = self.fc_2(combined)
        output = torch.cat((output1,output2),dim=1)
        output = self.fc(output)
        return output

<br>

## Training

In [54]:
class MultiRMSELoss(nn.Module):
    def __init__(self):
        super(MultiRMSELoss, self).__init__()

    def forward(self, output, target):
        loss1 = torch.sqrt(torch.mean((output[:,0]-target[:,0])**2))
        loss2 = torch.sqrt(torch.mean((output[:,1]-target[:,1])**2))
        loss = 0.5*loss1+0.5*loss2
        return loss

In [103]:
import gc
import numpy as np
import torch
import torch.nn as nn
import time

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
        
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        if self.path!='':
            torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

        
def train(
    model, criterion, optimizer, train_loader, valid_loader, epochs,
    early_stopping, device='cpu', scheduler=None, metric_period=1, 
    verbose=True, save_model_path = './mc/best_model.pt',
    use_best_model=True,
    inverse_transform=None,
):  
    seed_everything(CFG.SEED)
    model.to(device)

    best_loss  = 999999999
    best_epoch = 1
    best_model = None
    is_best    = np.nan
    
    start_time = time.time()
    epoch_s = time.time()
    
    model.train()
    for epoch in range(1, epochs+1):
        gc.collect()
        
        #model.train()
        train_loss = []
        for feat,img,target in train_loader:
            feat = feat.to(device)
            img = img.to(device)
            target = target.to(device)

            optimizer.zero_grad()
            output = model(img,feat)#.float()
            
#             if inverse_transform is not None:
#                 output = inverse_transform(output)
#                 target = inverse_transform(target)
            
            loss = criterion(output, target)
            loss.backward()  # Getting gradients
            optimizer.step() # Updating parameters

            train_loss.append(loss.item())

        if valid_loader is not None:
            valid_loss = validation(model, valid_loader, criterion, device, inverse_transform)
        else:
            valid_loss = loss
            
        epoch_e = time.time()
            
        if scheduler is not None:
            scheduler.step(valid_loss)

        # update the best epoch & best loss
        if (best_loss > valid_loss) | (epoch==1):
            best_epoch = epoch
            best_loss = valid_loss
            best_model = model
            is_best = 1
            torch.save(best_model.state_dict(), save_model_path)
        else:
            is_best = 0
            if not use_best_model:
                torch.save(best_model.state_dict(), save_model_path)
            
        # 결과물 printing
        if (verbose) & (epoch % metric_period == 0):
            mark = '*' if is_best else ' '
            epoch_str = str(epoch).zfill(len(str(epochs)))
            if valid_loader is not None:
                progress = '{}[{}/{}] loss: {:.5f}, val_loss: {:.5f}, best_epoch: {}, elapsed: {:.2f}s, total: {:.2f}s, remaining: {:.2f}s'\
                    .format(
                        mark,
                        epoch_str,
                        epochs,
                        np.mean(train_loss),
                        valid_loss,
                        best_epoch,
                        epoch_e-epoch_s,
                        epoch_e-start_time,
                        (epoch_e-epoch_s)*(epochs-epoch)/metric_period,
                    )
            else:
                progress = '{}[{}/{}] loss: {:.5f}, best_epoch: {}, elapsed: {:.2f}s, total: {:.2f}s, remaining: {:.2f}s'\
                    .format(
                        mark,
                        epoch_str,
                        epochs,
                        np.mean(train_loss),
                        best_epoch,
                        epoch_e-epoch_s,
                        epoch_e-start_time,
                        (epoch_e-epoch_s)*(epochs-epoch)/metric_period,
                    )
            epoch_s = time.time()
            print(progress)

        # early stopping 여부를 체크. 현재 과적합 상황 추적
        if early_stopping is not None:
            early_stopping(valid_loss, model)
            if early_stopping.early_stop:
                break

    return best_model

def validation(model, valid_loader, criterion, device, inverse_transform):
    valid_loss = []
    with torch.no_grad():
        for feat,img,target in valid_loader:
            feat = feat.to(device)
            img = img.to(device)
            target = target.to(device)
            
            output = model(img,feat)#.float()
            
            if inverse_transform is not None:
                output = torch.tensor(inverse_transform(output.cpu().numpy()),device=device)
                target = torch.tensor(inverse_transform(target.cpu().numpy()),device=device)

            loss = criterion(output, target)
            valid_loss.append(loss.item())

    return np.mean(valid_loss)

def predict(best_model,loader,device,inverse_transform):
    best_model.to(device)

    true_list = []
    pred_list = []
    with torch.no_grad():
        for feat,img,target in iter(loader):
            feat = feat.to(device)
            img = img.to(device)
            target = target.to(device)

            output = best_model(img,feat)
            
            if inverse_transform is not None:
                output = inverse_transform(output)
                target = inverse_transform(target)

            true_list.append(target)
            pred_list.append(output)

    trues = torch.cat(true_list,dim=0)
    preds = torch.cat(pred_list,dim=0)

    trues = trues.cpu().numpy()
    preds = preds.cpu().numpy()

    return trues, preds

In [104]:
feature_input_size = [feat for feat,smiles,target in train_dataset][0].shape[0]
output_size = 2
hidden_size = 64
dropout_rate = 0.0

In [105]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [106]:
model = MultiTaskModel(feature_input_size,output_size,hidden_size,dropout_rate)
criterion = MultiRMSELoss()
# optimizer= torch.optim.Adam(model.parameters(), lr=CFG.LEARNING_RATE)#, weight_decay=5e-4)
optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.LEARNING_RATE)
# optimizer = torch.optim.SGD(model.parameters(), lr=CFG.LEARNING_RATE, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-7, verbose=False)
# early_stopping = EarlyStopping(patience=10,verbose=False,path='')
early_stopping = None

In [107]:
torch.cuda.empty_cache()
gc.collect()

3342

In [108]:
best_model = train(
    model, criterion, optimizer, train_loader, val_loader,
    CFG.EPOCHS, early_stopping,
    device=device, scheduler=scheduler,
    metric_period=1, verbose=True,
    save_model_path='./mc/best_model.pt',
    use_best_model=False,
    inverse_transform=inverse_boxcox_transform,
)

*[0001/1024] loss: 5.56606, val_loss: 52.35971, best_epoch: 1, elapsed: 9.92s, total: 9.92s, remaining: 10144.85s
*[0002/1024] loss: 3.86467, val_loss: 38.39719, best_epoch: 2, elapsed: 9.62s, total: 19.68s, remaining: 9832.43s
*[0003/1024] loss: 2.94510, val_loss: 35.25707, best_epoch: 3, elapsed: 9.63s, total: 29.45s, remaining: 9829.87s
*[0004/1024] loss: 2.85123, val_loss: 34.82481, best_epoch: 4, elapsed: 9.68s, total: 39.29s, remaining: 9873.88s
*[0005/1024] loss: 2.72205, val_loss: 34.15099, best_epoch: 5, elapsed: 9.62s, total: 49.06s, remaining: 9804.42s
 [0006/1024] loss: 2.63933, val_loss: 34.18087, best_epoch: 5, elapsed: 9.62s, total: 58.82s, remaining: 9794.48s
 [0007/1024] loss: 2.55869, val_loss: 35.31099, best_epoch: 5, elapsed: 9.62s, total: 68.57s, remaining: 9779.19s
 [0008/1024] loss: 2.47819, val_loss: 34.44461, best_epoch: 5, elapsed: 9.62s, total: 78.33s, remaining: 9774.53s
 [0009/1024] loss: 2.30068, val_loss: 34.86433, best_epoch: 5, elapsed: 9.62s, total: 88

KeyboardInterrupt: 

<br>

## Inference

In [ ]:
best_model = MultiTaskModel(feature_input_size,output_size,hidden_size,dropout_rate)
best_model.load_state_dict(torch.load('./mc/best_model.pt'))

In [ ]:
tr_true, tr_pred = predict(best_model,train_loader,device,inverse_transform)
va_true, va_pred = predict(best_model,val_loader,device,inverse_transform)

In [ ]:
(MultiRMSELoss()(torch.tensor(tr_true),torch.tensor(tr_pred)),
 MultiRMSELoss()(torch.tensor(va_true),torch.tensor(va_pred)))

In [ ]:
tr_true[:10].round(1), tr_pred[:10].round(1)
# va_true[:10].round(1), va_pred[:10].round(1)

In [ ]:
def abline(intercept,slope):
    axes = plt.gca()
    x_vals = np.array(axes.get_xlim())
    y_vals = intercept + slope * x_vals
    plt.plot(x_vals, y_vals, linestyle='--', color='red')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig = plt.figure(figsize=(15,7))
fig.add_subplot(121)
sns.scatterplot(x=tr_true[:,0],y=tr_pred[:,0])
abline(0,1)
plt.xlabel('true')
plt.ylabel('pred')
plt.grid()
fig.add_subplot(122)
sns.scatterplot(x=tr_true[:,1],y=tr_pred[:,1])
abline(0,1)
plt.xlabel('true')
plt.ylabel('pred')
plt.grid()
plt.suptitle('train',fontsize=20)
plt.tight_layout()
plt.show()

fig = plt.figure(figsize=(15,7))
fig.add_subplot(121)
sns.scatterplot(x=va_true[:,0],y=va_pred[:,0])
abline(0,1)
plt.xlabel('true')
plt.ylabel('pred')
plt.grid()
fig.add_subplot(122)
sns.scatterplot(x=va_true[:,1],y=va_pred[:,1])
abline(0,1)
plt.xlabel('true')
plt.ylabel('pred')
plt.grid()
plt.suptitle('validation',fontsize=20)
plt.tight_layout()
plt.show()